Inputs are a hypothesis (with multiple individual vectors) and a set of simulataneous detections

Output is the set of new hypotheses

In [17]:
import numpy as np
import itertools
import igraph

In [28]:
bg_prob = 0.1

In [72]:
hyp = [[0, np.nan], [np.nan, 1]]
score = 1


dets = [2, 3]


Create the case where all detections are new individuals

In [73]:
new_scen = [h + [np.nan] for h in hyp] + [list(np.empty(len(hyp[0])) * np.nan) + [d] for d in dets]

Create branches for the repeat scenarios for each existing branch and the new detections

In [74]:
repeat_scen = []
for h in hyp:
    for d in dets:
        repeat_scen.append(h + [d])

Append the branches from the two to create all the branches for this step

In [75]:
new_scen

[[0, nan, nan], [nan, 1, nan], [nan, nan, 2], [nan, nan, 3]]

In [76]:
repeat_scen

[[0, nan, 2], [0, nan, 3], [nan, 1, 2], [nan, 1, 3]]

In [77]:
all_branches = new_scen + repeat_scen

In [78]:
all_branches

[[0, nan, nan],
 [nan, 1, nan],
 [nan, nan, 2],
 [nan, nan, 3],
 [0, nan, 2],
 [0, nan, 3],
 [nan, 1, 2],
 [nan, 1, 3]]

In [79]:
def get_last_detection_id(br):
    return np.array(br)[np.logical_not(np.isnan(br))][-1]

In [80]:
def score_new_branches(branches, bg_prob, probz):
    scores = [0] * len(branches)
    
    for i, br in branches:
        # if it's just the padded branches form the existing hypothesis then score is zero
        if br[-1] == np.nan:
            scores[i] = 0
        # if it ends in a new detections, but all nans otherwise then score is the background
        elif np.sum(np.logical_not(np.isnan(d[:-1]))) == 0:
            scores[i] = bg_prob
        # otherwise, calculate the repeat prob
        else:
            current_det = br[-1]
            last_det = get_last_detection_id(br[:-1])
            scores[i] = probz[int(last_det), int(current_det)]
            
    return scores
            

In [95]:
def make_branches_multi(hypothesis, detection_ids):
    # Create the case where all detections are new individuals
    new_scen = [h + [np.nan] for h in hyp] + [list(np.empty(len(hyp[0])) * np.nan) + [d] for d in dets]
    
    repeat_scen = []
    for h in hyp:
        for d in dets:
            repeat_scen.append(h + [d])
            
    all_branches = new_scen + repeat_scen
    return all_branches

In [81]:
def make_tree_compat(trees):
    trees = np.array(trees)
    tree_compat = np.zeros((len(trees), len(trees)))

    for step in trees.transpose():
        unique, counts = np.unique(step, return_counts=True)
        reps = unique[np.where(counts > 1)]
        for r in reps:
            if r >= 0:
                whr = np.where(step == r)
                for pair in itertools.permutations(whr[0], r=2):
                    tree_compat[pair] = 1
    return tree_compat

def make_hypotheses_from_trees(trees):

    tree_compat = make_tree_compat(trees)
    g = igraph.Graph()
    nverts = len(trees)
    g.add_vertices(nverts)
    labels = []
    edges = []
    for i in range(nverts):
        labels.append(str(i))
        for ed in np.where(tree_compat[i] ==1)[0]:
            edges.append((i,ed))


    g.vs["label"] = labels

    g.add_edges(edges)
    hyps =  g.maximal_independent_vertex_sets()
    return hyps


In [82]:
new_hyp_inds = make_hypotheses_from_trees(all_branches)

In [83]:
new_hyp_inds

[(0, 1, 2, 3), (0, 2, 7), (0, 3, 6), (1, 2, 5), (5, 6), (1, 3, 4), (4, 7)]

### Score the new hypotheses


In [85]:
import pandas as pd
import matplotlib.pyplot as plt

detections = pd.DataFrame()
detections["id"] = [0, 1, 2, 3]
detections["time"] = [0, 1, 2, 3]
detections["object_location"] = [0, 1, 2, 3]

In [87]:
fauna_step = 0.2
background_prob = 0.2

In [92]:
from src.hypotheses import build_and_score_hypotheses, generate_hypothesis_matrix

In [93]:
probz = generate_hypothesis_matrix(detections, fauna_step)

/home/nader/Projects/thesis/benthic_targets/src/hypotheses.py:83: RuntimeWarning: invalid value encountered in true_divide
  z_stat = np.divide(dists, faunastep * Dtime)


In [94]:
probz

array([[           nan, 5.73303144e-07, 5.73303144e-07, 5.73303144e-07],
       [5.73303144e-07,            nan, 5.73303144e-07, 5.73303144e-07],
       [5.73303144e-07, 5.73303144e-07,            nan, 5.73303144e-07],
       [5.73303144e-07, 5.73303144e-07, 5.73303144e-07,            nan]])

In [89]:
hyps0, scores0 = build_and_score_hypotheses(detections, fauna_step, background_prob)

/home/nader/Projects/thesis/benthic_targets/src/hypotheses.py:83: RuntimeWarning: invalid value encountered in true_divide
  z_stat = np.divide(dists, faunastep * Dtime)


In [90]:
hyps0

[[[0, nan, nan, nan],
  [nan, 1, nan, nan],
  [nan, nan, 2, nan],
  [nan, nan, nan, 3]],
 [[0, nan, nan, 3], [nan, 1, nan, nan], [nan, nan, 2, nan]],
 [[nan, 1, nan, 3], [0, nan, nan, nan], [nan, nan, 2, nan]],
 [[nan, nan, 2, 3], [0, nan, nan, nan], [nan, 1, nan, nan]],
 [[0, nan, 2, nan], [nan, 1, nan, nan], [nan, nan, nan, 3]],
 [[0, nan, 2, 3], [nan, 1, nan, nan]],
 [[nan, 1, nan, 3], [0, nan, 2, nan]],
 [[nan, 1, 2, nan], [0, nan, nan, nan], [nan, nan, nan, 3]],
 [[nan, 1, 2, 3], [0, nan, nan, nan]],
 [[0, nan, nan, 3], [nan, 1, 2, nan]],
 [[0, 1, nan, nan], [nan, nan, 2, nan], [nan, nan, nan, 3]],
 [[0, 1, nan, 3], [nan, nan, 2, nan]],
 [[nan, nan, 2, 3], [0, 1, nan, nan]],
 [[0, 1, 2, nan], [nan, nan, nan, 3]],
 [[0, 1, 2, 3]]]

In [91]:
scores0

[0.8,
 0.6000005733031439,
 0.6000005733031439,
 0.6000005733031439,
 0.6000005733031439,
 0.4000011466062877,
 0.4000011466062877,
 0.6000005733031439,
 0.4000011466062877,
 0.4000011466062877,
 0.6000005733031439,
 0.4000011466062877,
 0.4000011466062877,
 0.4000011466062877,
 0.20000171990943155]